###  Mount Google Drive

In [1]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3


Mounted at /content/drive


### Clone Codebase

In [ ]:
! mkdir -p /content/drive/MyDrive/COMP4901B-Homework3
! cd /content/drive/MyDrive/COMP4901B-Homework3 && git clone https://github.com/hkust-nlp/COMP4901B-LLMs.git

Cloning into 'COMP4901B-LLMs'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 232 (delta 40), reused 24 (delta 24), pack-reused 169 (from 1)
Receiving objects: 100% (232/232), 798.55 KiB | 1.87 MiB/s, done.
Resolving deltas: 100% (87/87), done.


### Download Dataset

In [2]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/setup.sh

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 153.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 111.3 MB/s eta 

### Q1 code evaulation for q1

In [ ]:
! cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3/ && python tests/test_format_prompts.py


SANITY CHECK: format_prompts()

This script tests your implementation of the format_prompts function.
It compares your outputs with expected reference outputs.


TEST 2: Few-shot without chat template
2025-11-13 09:43:54.067356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763027034.487824    6130 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763027034.594252    6130 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763027035.283583    6130 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763027035.283628    6130 computation_placer.cc:

### Q4 Start The Tuning

#### Default

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B

GSM8K Self-Improving Training Pipeline
Run Name: first-run
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/first-run
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 1
  Gradient Accumulation Steps: 128 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/first-run/iteration_0/inference.jsonl

2025-11-14 11:42:24.446372: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn the

In [ ]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/first-run/models/model_iter_1-merged \
    --output_dir results/default/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/first-run/models/model_iter_1-merged
Output Directory: results/default/rollout1
Run Name: model_iter_1-merged_20251115_070341
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/default/rollout1/model_iter_1-merged_20251115_070341_inference.jsonl

2025-11-15 07:03:48.709790: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 07:03:48.727304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered

In [ ]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/first-run/models/model_iter_1-merged \
    --output_dir results/default/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/first-run/models/model_iter_1-merged
Output Directory: results
Run Name: model_iter_1-merged_20251115_060728
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/model_iter_1-merged_20251115_060728_inference.jsonl

2025-11-15 06:07:32.677374: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-15 06:07:32.694729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763186852.716271   

#### Decrease batch size (64)

In [5]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --total_batch_size 64 --run_name batch_64 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: batch_64
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/batch_64
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 64
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 16 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/batch_64/iteration_0/inference.jsonl

2025-11-17 09:14:53.547755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off

In [6]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/batch_64/models/model_iter_1-merged \
    --output_dir results/batch_64/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/batch_64/models/model_iter_1-merged
Output Directory: results/batch_64/rollout1
Run Name: model_iter_1-merged_20251117_100606
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/batch_64/rollout1/model_iter_1-merged_20251117_100606_inference.jsonl

2025-11-17 10:06:12.186882: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:06:12.204702: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registere

In [7]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/batch_64/models/model_iter_1-merged \
    --output_dir results/batch_64/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/batch_64/models/model_iter_1-merged
Output Directory: results/batch_64/rollout8
Run Name: model_iter_1-merged_20251117_100949
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/batch_64/rollout8/model_iter_1-merged_20251117_100949_inference.jsonl

2025-11-17 10:09:54.161883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:09:54.180482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been register

#### Increase batch size (256)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --total_batch_size 256 --run_name batch_256 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: batch_256
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/batch_256
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 256
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 64 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/batch_256/iteration_0/inference.jsonl

2025-11-15 09:11:10.826084: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them

In [8]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/batch_256/models/model_iter_1-merged \
    --output_dir results/batch_256/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/batch_256/models/model_iter_1-merged
Output Directory: results/batch_256/rollout1
Run Name: model_iter_1-merged_20251117_102749
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/batch_256/rollout1/model_iter_1-merged_20251117_102749_inference.jsonl

2025-11-17 10:27:54.164418: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:27:54.183398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regist

In [9]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/batch_256/models/model_iter_1-merged \
    --output_dir results/batch_256/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/batch_256/models/model_iter_1-merged
Output Directory: results/batch_256/rollout8
Run Name: model_iter_1-merged_20251117_103146
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/batch_256/rollout8/model_iter_1-merged_20251117_103146_inference.jsonl

2025-11-17 10:31:51.284543: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:31:51.303147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regis

#### Increase queries (3000)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --n_queries 3000 --run_name queries_3000 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: queries_3000
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/queries_3000
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 3000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/queries_3000/iteration_0/inference.jsonl

2025-11-15 08:37:09.974081: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To 

In [10]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_3000/models/model_iter_1-merged \
    --output_dir results/queries_3000/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/queries_3000/models/model_iter_1-merged
Output Directory: results/queries_3000/rollout1
Run Name: model_iter_1-merged_20251117_104943
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/queries_3000/rollout1/model_iter_1-merged_20251117_104943_inference.jsonl

2025-11-17 10:49:48.553948: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:49:48.572644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already be

In [11]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_3000/models/model_iter_1-merged \
    --output_dir results/queries_3000/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/queries_3000/models/model_iter_1-merged
Output Directory: results/queries_3000/rollout8
Run Name: model_iter_1-merged_20251117_105342
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/queries_3000/rollout8/model_iter_1-merged_20251117_105342_inference.jsonl

2025-11-17 10:53:47.035518: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 10:53:47.054396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already b

#### Decrease queries (1000)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --n_queries 1000 --run_name queries_1000 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: queries_1000
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/queries_1000
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 1000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/queries_1000/iteration_0/inference.jsonl

2025-11-15 09:33:20.536588: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To 

In [12]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_1000/models/model_iter_1-merged \
    --output_dir results/queries_1000/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/queries_1000/models/model_iter_1-merged
Output Directory: results/queries_1000/rollout1
Run Name: model_iter_1-merged_20251117_111139
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/queries_1000/rollout1/model_iter_1-merged_20251117_111139_inference.jsonl

2025-11-17 11:11:44.166886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:11:44.185830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already be

In [13]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/queries_1000/models/model_iter_1-merged \
    --output_dir results/queries_1000/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/queries_1000/models/model_iter_1-merged
Output Directory: results/queries_1000/rollout8
Run Name: model_iter_1-merged_20251117_111610
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/queries_1000/rollout8/model_iter_1-merged_20251117_111610_inference.jsonl

2025-11-17 11:16:15.452220: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:16:15.470443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already b

#### Increase lr (4e-5)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --learning_rate 4e-5 --run_name "lr_4e-5" --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: lr_4e-5
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/lr_4e-5
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 4e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/lr_4e-5/iteration_0/inference.jsonl

2025-11-15 13:29:18.218469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, 

In [14]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_4e-5/rollout1" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

bash: scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged: No such file or directory


In [15]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_4e-5/rollout8" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

bash: scripts/run_gsm8k_eval.sh ckpt/lr_4e-5/models/model_iter_1-merged: No such file or directory


#### Decrease lr (1e-5)

In [ ]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --learning_rate 1e-5 --run_name "lr_1e-5" --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: lr_1e-5
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/lr_1e-5
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 1e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 64
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/lr_1e-5/iteration_0/inference.jsonl

2025-11-15 13:53:32.948355: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, 

In [16]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_1e-5/rollout1" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

bash: scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged: No such file or directory


In [17]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash "scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged" \
    --output_dir "results/lr_1e-5/rollout8" \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

bash: scripts/run_gsm8k_eval.sh ckpt/lr_1e-5/models/model_iter_1-merged: No such file or directory


#### Increase LoRA rank (128)

In [3]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --lora_r 128 --run_name rank_128 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: rank_128
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/rank_128
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 128
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/rank_128/iteration_0/inference.jsonl

2025-11-17 07:28:52.709091: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them o

In [18]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_128/models/model_iter_1-merged \
    --output_dir results/rank_128/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/rank_128/models/model_iter_1-merged
Output Directory: results/rank_128/rollout1
Run Name: model_iter_1-merged_20251117_113406
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/rank_128/rollout1/model_iter_1-merged_20251117_113406_inference.jsonl

2025-11-17 11:34:10.648773: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:34:10.667841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registere

In [19]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_128/models/model_iter_1-merged \
    --output_dir results/rank_128/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/rank_128/models/model_iter_1-merged
Output Directory: results/rank_128/rollout8
Run Name: model_iter_1-merged_20251117_113759
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/rank_128/rollout8/model_iter_1-merged_20251117_113759_inference.jsonl

2025-11-17 11:38:04.510166: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:38:04.528546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been register

#### Decrease LoRA rank (32)

In [4]:
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && \
export WANDB_API_KEY="cdb87cd68d7a18cd6d0a4e34d52b1ef4b41c20e3" && \
export WANDB_PROJECT="COMP4901B-Homework3" && \
export CUDA_VISIBLE_DEVICES="0" && \
bash scripts/self_train_gsm8k.sh Qwen3-0.6B --lora_r 32 --run_name rank_32 --batch_size_per_dev 4

GSM8K Self-Improving Training Pipeline
Run Name: rank_32
Initial Model: Qwen3-0.6B
Pipeline Directory: ckpt/rank_32
Number of Iterations: 1

Inference Settings:
  Dataset Split: train
  Number of Queries: 2000
  Max Tokens: 512
  Temperature: 1.0
  Top-p: 1
  Top-k: -1
  Number of Rollouts: 8
  Tensor Parallel: 1
  Mode: Zero-shot
  Chat Template: Enabled
  Thinking Mode: Disabled

Training Settings:
  Learning Rate: 2e-5
  Total Batch Size: 128
  Batch Size Per Device: 4
  Gradient Accumulation Steps: 32 (auto-calculated)
  Number of Epochs: 1
  Save Steps: 30
  LoRA Rank: 32
  Model Max Length: 712 (auto-calculated: MAX_TOKENS + 200)

ITERATION 0 / 0
Model: Qwen3-0.6B

[Step 1/4] Running inference...
Output: ckpt/rank_32/iteration_0/inference.jsonl

2025-11-17 08:22:23.224842: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, 

In [20]:
# Evaluation of iteration 1 rollout 1
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_32/models/model_iter_1-merged \
    --output_dir results/rank_32/rollout1 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 1

GSM8K Evaluation Pipeline
Model: ckpt/rank_32/models/model_iter_1-merged
Output Directory: results/rank_32/rollout1
Run Name: model_iter_1-merged_20251117_115602
Dataset Split: test
Number of Queries: All
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 1
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/rank_32/rollout1/model_iter_1-merged_20251117_115602_inference.jsonl

2025-11-17 11:56:07.546182: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:56:07.564798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E

In [21]:
# Evaluation of iteration 1 rollout 8
!cd /content/drive/MyDrive/COMP4901B-Homework3/COMP4901B-LLMs/assignment3 && bash scripts/run_gsm8k_eval.sh ckpt/rank_32/models/model_iter_1-merged \
    --output_dir results/rank_32/rollout8 \
    --temperature 0.6 \
    --max_tokens 512 \
    --top_p 0.95 \
    --top_k 20 \
    --n_rollouts 8 \
    --n_queries 2000

GSM8K Evaluation Pipeline
Model: ckpt/rank_32/models/model_iter_1-merged
Output Directory: results/rank_32/rollout8
Run Name: model_iter_1-merged_20251117_115942
Dataset Split: test
Number of Queries: 2000
Max Tokens: 512
Temperature: 0.6
Top-p: 0.95
Top-k: 20
Number of Rollouts: 8
Tensor Parallel: 1
Mode: Zero-shot
Chat Template: Enabled
Thinking Mode: Disabled

[1/2] Running inference...
Output will be saved to: results/rank_32/rollout8/model_iter_1-merged_20251117_115942_inference.jsonl

2025-11-17 11:59:46.658805: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-17 11:59:46.677468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
